In [ ]:
# Load library imports
import sys
import torch
import random
import joblib
import logging
import importlib
import numpy as np
import pandas as pd

# Load project Imports
from src.utils.config_loader import load_project_config, deep_format, expanduser_tree
from src.data_ingestion.gwl_data_ingestion import process_station_coordinates, \
    fetch_and_process_station_data, download_and_save_station_readings
from src.preprocessing.gwl_preprocessing import load_timeseries_to_dict, outlier_detection, \
    resample_timestep_average, remove_spurious_data, handle_short_gaps
from src.archive.gap_imputation_copy import synthetic_gap_imputation_validation, define_catchment_size, \
    calculate_station_distances, calculate_station_correlations, score_station_proximity

In [ ]:
# Set up logger config
logging.basicConfig(
    level=logging.INFO,
   format='%(levelname)s - %(message)s',
#    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set up logger for file and load config file for paths and params
logger = logging.getLogger(__name__)
config = load_project_config(config_path="config/project_config.yaml")
notebook = True

# Set up root directory paths in config
raw_data_root = config["global"]["paths"]["raw_data_root"]
results_root = config["global"]["paths"]["results_root"]

# Reformat config roots
config = deep_format(
    config,
    raw_data_root=raw_data_root,
    results_root=results_root
)
config = expanduser_tree(config)

In [ ]:
# Set up seeding to define global states
random_seed = config["global"]["pipeline_settings"]["random_seed"]
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define notebook demo catchment
catchments_to_process = config["global"]["pipeline_settings"]["catchments_to_process"]
catchment = catchments_to_process[0]
run_defra_API_calls = config["global"]["pipeline_settings"]["run_defra_api"]  # True to run API calls

logger.info(f"Show Notebook Outputs: {notebook}")
logger.info(f"Notebook Demo Catchment: {catchment.capitalize()}")

### DATA INGESTION ###

Load gwl station list with grid references and convert grid references to easting, northing, longitude and latitude form for plotting and data alignment.

In [ ]:
# --- Process Catchment Stations List ----
stations_with_coords_df = process_station_coordinates(
    os_grid_squares=config["global"]["paths"]["gis_os_grid_squares"],
    station_list_input=config[catchment]["paths"]["gwl_station_list"],
    station_list_output=config[catchment]["paths"]["gwl_station_list_with_coords"],
    catchment=catchment
)

logger.info(f"Pipeline step 'Process Station Coordinates for {catchment}' complete.\n")

**API Documentation notes:**

1. The API calls that return readings data have a soft limit of 100,000 rows per-call which can be overridden by setting a _limit parameter. There is a hard limit of 2,000,000 rows, which cannot be overridden.
2. The primary identifier for most stations uses a GUID style identifier called an SUID. These are used in the URL for the station and given as the value of the notation property in the station metadata.  
    a. Wiski identifier (wiskiID) is also available for my subset of stations and data type  
3. All monitoring stations can be filtered by name, location and other parameters. See https://environment.data.gov.uk/hydrology/doc/reference#stations-summary for full metadata details

In [ ]:
if run_defra_API_calls:
    # Retrieve gwl monitoring station metadata and measures from DEFRA API
    stations_with_metadata_measures = fetch_and_process_station_data(
        stations_df=stations_with_coords_df,
        base_url=config["global"]["paths"]["defra_station_base_url"],
        output_path=config[catchment]["paths"]["gwl_station_metadata_measures"]
    )

    logger.info(f"Pipeline step 'Pull Hydrological Station Metadata for {catchment}' complete.\n")

    stations_with_metadata_measures.head()

In [ ]:
if run_defra_API_calls:
    download_and_save_station_readings(
        stations_df=stations_with_metadata_measures,
        start_date=config["global"]["data_ingestion"]["api_start_date"],
        end_date=config["global"]["data_ingestion"]["api_end_date"],
        gwl_data_output_dir=config[catchment]["paths"]["gwl_data_output_dir"]
    )

    logger.info(f"All timeseries groundwater level data saved for {catchment} catchment.")
    
else:
    
    loaded_csv_path = config[catchment]["paths"]["gwl_station_metadata_measures"]
    stations_with_metadata_measures = pd.read_csv(loaded_csv_path)

### PREPROCESSING ###

Remove stations with insufficient data and clean ts data from outliers and incorrect measurements. Interpolate between small data gaps using rational spline.

1. Load station df's into dict, dropping catchments with insufficient data

In [ ]:
# Load timeseries CSVs from API into reference dict
gwl_time_series_dict = load_timeseries_to_dict(
    stations_df=stations_with_metadata_measures,
    col_order=config["global"]["data_ingestion"]["col_order"],
    data_dir=config[catchment]["paths"]["gwl_data_output_dir"],
    inclusion_threshold=config[catchment]["preprocessing"]["inclusion_threshold"],
    station_list_output=config[catchment]["paths"]["gwl_station_list_output"],
    catchment=catchment
)

logger.info(f"All timeseries data converted to dict for {catchment} catchment.\n")

2. Remove outlying and incorrect data points

In [ ]:
for station_name, df in gwl_time_series_dict.items():
    gwl_time_series_dict[station_name] = remove_spurious_data(
        target_df=df,
        station_name=station_name,
        path=config[catchment]["visualisations"]["ts_plots"]["time_series_gwl_output"],
        pred_frequency=config["global"]["pipeline_settings"]["prediction_resolution"],
        notebook=notebook
    )

In [ ]:
run_outlier_processing = config["global"]["pipeline_settings"]["run_outlier_detection"]

if run_outlier_processing:
    # run outlier detection and processing
    processed_gwl_time_series_dict = outlier_detection(
        gwl_time_series_dict=gwl_time_series_dict,
        output_path=config[catchment]["visualisations"]["ts_plots"]["time_series_gwl_output"],
        dpi=config[catchment]["visualisations"]["ts_plots"]["dpi_save"],
        dict_output=config[catchment]["paths"]["gwl_outlier_dict"],
        notebook=notebook
    )

3. Aggregate to timestep frequency

In [ ]:
if not run_outlier_processing:
    input_dict = config[catchment]["paths"]["gwl_outlier_dict"]
    processed_gwl_time_series_dict = joblib.load(input_dict)

timestep_data = resample_timestep_average(
    gwl_data_dict=processed_gwl_time_series_dict,
    start_date=config["global"]["data_ingestion"]["api_start_date"],
    end_date=config["global"]["data_ingestion"]["api_end_date"],
    path=config[catchment]["visualisations"]["ts_plots"]["time_series_gwl_output"],
    pred_frequency=config["global"]["pipeline_settings"]["prediction_resolution"],
    notebook=notebook
)

4. Interpolate across small gaps in the ts data using rational spline or PCHIP - try both (& define threshold n/o missing time steps for interpolation eligibility) + Add binary interpolation flag column

In [ ]:
timestep_data, gaps_list, station_max_gap_lengths_calculated = handle_short_gaps(
    timestep_data=timestep_data,
    path=config[catchment]["visualisations"]["ts_plots"]["time_series_gwl_output"],
    max_steps=config["global"]["data_ingestion"]["max_interp_length"],
    start_date=config["global"]["data_ingestion"]["api_start_date"],
    end_date=config["global"]["data_ingestion"]["api_end_date"],
    pred_frequency=config["global"]["pipeline_settings"]["prediction_resolution"],
    notebook=notebook
)

Handle large gaps

In [ ]:
# Ensure all names match formatting to avoid mismatches
def _clean_name(s):
    s_clean = s.lower().replace(" ", "_")
    return s_clean

timestep_data = {_clean_name(key): value for key, value in timestep_data.items()}
gaps_list = [_clean_name(station) for station in gaps_list]

In [ ]:
# build dist + corr matrices
spatial_df = pd.read_csv(config[catchment]["paths"]["gwl_station_list_output"])
spatial_df["station_name"] = spatial_df["station_name"].str.lower().str.replace(" ", "_")

large_catchment = define_catchment_size(
    spatial_df, catchment,
    threshold_m=config[catchment]["preprocessing"]["large_catchment_threshold_m"]
)
distance_matrix = calculate_station_distances(
    spatial_df,
    use_haversine=large_catchment,
    radius=config["global"]["preprocessing"]["radius"],
)
distance_matrix = np.round(distance_matrix, 2)

correlation_matrix = calculate_station_correlations(timestep_data, catchment)

# 2) score donors and threshold
filtered_scores = score_station_proximity(
    df_dist=timestep_data,
    gaps_list=gaps_list,  # stations to validate
    correlation_matrix=correlation_matrix,
    distance_matrix=distance_matrix,
    k_decay=config[catchment]["preprocessing"]["dist_corr_score_k_decay"],
    output_path=config[catchment]["visualisations"]["corr_dist_score_scatters"],
    threshold=config[catchment]["preprocessing"]["dist_corr_score_threshold"],
)


In [ ]:
# Internal keys  ->  Pretty labels in the paper
methods = ["pchip", "spline", "linear", "donor_station"]
gap_lengths = [7,14,30,60,90,120,150,180,250,365]
labels = {"pchip": "PCHIP", "spline": "Spline",
          "linear": "Linear", "donor_station": "Donor-station"}

rows = []
for m in methods:
    for L in gap_lengths:
        
        # run once per call to collect results
        (_, _, _, summary_df) = synthetic_gap_imputation_validation(
            df_dict_original=timestep_data,
            gaps_list=gaps_list,
            min_around=config["global"]["preprocessing"]["min_data_points_around_gap"],
            predefined_large_gap_lengths=[L],
            max_imputation_length_threshold=L,
            filtered_scores=filtered_scores,
            validation_plot_path="results/figures/eden/imputation_validation/validation",
            imputation_plot_path="results/figures/eden/imputation_validation/imputation",
            pred_frequency=config["global"]["pipeline_settings"]["prediction_resolution"],
            station_max_gap_lengths=station_max_gap_lengths_calculated,
            random_seed=config["global"]["pipeline_settings"]["random_seed"],
            imputation_method=("weighted_average" if m == "donor_station" else m),
            distance_matrix=distance_matrix,   # <— NEW
            k_decay=config[catchment]["preprocessing"]["dist_corr_score_k_decay"],  # <— NEW
        )
        
        # summary_df of metrics
        summary_df = summary_df.copy()
        summary_df["method"] = labels[m]
        summary_df["gap_days"] = L
        rows.append(summary_df)

import pandas as pd
tidy = pd.concat(rows, ignore_index=True)
tidy = tidy[["gap_days", "method", "RMSE_mean", "MAE_mean", "KGE_mean"]]

# Pivot to match table layout
wide = (tidy
        .set_index(["gap_days","method"])
        .sort_index()
        .unstack("method"))  # MultiIndex columns: (metric, method)

# Reorder the column MultiIndex as in headings
method_order = ["PCHIP", "Spline", "Linear", "Donor-station"]
metric_order = ["RMSE_mean", "MAE_mean", "KGE_mean"]

# Build new ordered set of columns
new_cols = []
for meth in method_order:
    for metric in metric_order:
        new_cols.append((metric, meth))

# reindex cols to order and round to table accuracy
wide = wide.reindex(columns=pd.MultiIndex.from_tuples(new_cols))
wide = wide.round(3)

print(wide)